In [ ]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

In [ ]:
# from copy import deepcopy
fracSaturTh = .05
# movieFreq = 2
cmap = plt.cm.Greys
cmap.set_bad("lime")
pd.set_option('display.max_rows', 100)

In [ ]:
plt.rcParams["animation.embed_limit"] = 200

In [ ]:
recFile = "/data/Johannes/2020_08_10/Experiment004h.nd2"

In [ ]:
rec = Recording(recFile)
rec.metadata

In [ ]:
ser = rec.metadata.Name[0]
# ser = "all"

In [ ]:
rec.metadata.loc[0,"SizeT"]/rec.metadata.loc[0,"Frequency"]/60

In [ ]:
rec.import_series(ser, isLineScan=False, onlyMeta=False, 
#                   restrict=(0,5160)
                 )

In [ ]:
metadata = rec.Series[ser]['metadata']

In [ ]:
movie = cmovie(rec.Series[ser]['data'],fr = metadata.Frequency)

In [ ]:
movie.shape

In [ ]:
movie.fr

In [ ]:
5/metadata.pxSize

In [ ]:
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph, plotRoi_to_be_connected

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
from islets.Recording import saveMovie

In [ ]:
saveMovie(movie[len(movie)*2//3:],"embed", maxFreq=1)

In [ ]:
Nsatur = (movie==movie.max()).sum(0)

In [ ]:
toAnull = np.where(Nsatur>len(movie)*fracSaturTh)

In [ ]:
movie[(slice(None), )+toAnull] = 0

In [ ]:
dmovie = np.diff(movie)

In [ ]:
from cv2 import GaussianBlur

In [ ]:
# image = pc_low+pc_high
# image = movie.mean(0)
image = dmovie.std(0)
image = GaussianBlur(image,(3,3),-1)

In [ ]:
im_th = get_sep_th(image.flatten(),log=False,plot=True)

In [ ]:
plt.imshow(movie.mean(0))

In [ ]:
# 6/metadata.pxSize

In [ ]:
regions = Regions(movie,
                  gSig_filt=(4,5),
                  diag=True,
                  processes=1
                 )

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.subplot(111)
regions.plotEdges(ax=ax)
regions.plotPeaks(ax=ax)

In [ ]:
# while True:
#     maxSize = np.percentile(regions.df["size"],80)
#     sizeTh = maxSize/2
#     peak2bnd = getPeak2BoundaryDF(regions.df)
#     df = peak2bnd.query("dist<1.")[["i","j"]]
#     if len(df)==0: break
#     df=df[df.i.isin(regions.df.loc[df.i].query(f"size<{sizeTh}").index)]
#     df=df[df.j.isin(regions.df.loc[df.j].query(f"size<{maxSize}").index)]
#     if len(df)==0: break
#     plt.figure(figsize=(10,10))
#     ax = plt.subplot(111)
#     regions.plotEdges(ax=ax,imkw_args={"cmap":"Greys","vmax":3}, color="blue")
#     regions.plotPeaks(ax=ax, color="darkred")
#     gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=True, ax=ax)
#     dropped = mergeBasedOnGraph(gRois,regions)
#     plt.show()
# #     if dropped == 0: break

In [ ]:
# saveRois(
#     regions, 
#     f"{recFile}_analysis/{ser}", 
#     filename="srdjan_handcurated",
#     col=[],
#     formats=["vienna"]
#         )

In [ ]:
# for sizeTh in range(2,6):
#     if sum(regions.df['size']<sizeTh)<1: continue
#     th = get_sep_th(regions.df.query(f"size<{sizeTh}").interest,log=True, plot=True)
#     drop = regions.df.query(f"size<{sizeTh} and interest<{th}").index
#     regions.df.drop(index=drop, inplace=True)
#     regions.update()
#     peak2bnd = getPeak2BoundaryDF(regions.df)
#     df = peak2bnd.query("dist<1.")[["i","j"]]
#     # if len(df)==0: break
#     df=df[df.i.isin(regions.df.loc[df.i].query(f"size<{sizeTh}").index)]
#     gRois = getGraph_of_ROIs_to_Merge(df,regions,plot=True)
#     dropped = mergeBasedOnGraph(gRois,regions)
#     # if dropped == 0: break
#     plt.show()

#### import saved

In [ ]:
pickleFilename = f"{recFile}_analysis/{ser}/"+[f for f in os.listdir(f"{recFile}_analysis/{ser}") if "handcur" in f][0]
with open(pickleFilename, "rb") as f:
    regions = pickle.load(f)

In [ ]:
n0 = len(movie)//5
regions.update(movie[n0:])

In [ ]:
regions.Freq 

In [ ]:
regions.time[0]

In [ ]:
regions.time += n0/regions.Freq 

### Rest

In [ ]:
regions.update(movie[:len(movie)*3//4])

In [ ]:
regions.calcTraces()

In [ ]:
regions.import_protocol(f"{recFile}_analysis/{ser}/"+[f for f in os.listdir(f"{recFile}_analysis/{ser}/") if "protocol" in f][0])

In [ ]:
for c in regions.df.columns:
    if "_" in c:
        del regions.df[c]

try: del regions.showTime
except: pass

In [ ]:
for ts in [1,3,10,30,100,300]:
    regions.fast_filter_traces(ts,)

from scipy.stats import rankdata

In [ ]:
regions.df["zInterest"] = np.max([rankdata((np.vstack(regions.df[c])>3.5).mean(1)) \
                                  for c in regions.df.columns if "zScore" in c],axis=0)

In [ ]:
regions.detrend_traces()

In [ ]:
regions.examine2(
    startShow=regions.df.sort_values("zInterest", ascending=False).index[:200]
)

In [ ]:
regions.df.loc[486]

In [ ]:
from islets.fitting import myShape, myLogNorm
from collections import namedtuple
from scipy.optimize import minimize

In [ ]:
from islets.numeric import guessDecayPars

In [ ]:
def toMin(x, t_, y_, shape=myShape):
    yf_ = shape(t_,*x,fillnan=False)
    return ((yf_-y_)**2).sum()

In [ ]:
Parameters = namedtuple("Parameters", ['decay', 'half_width', 'amplitude', 'loc', 'offset', 'end'])

In [ ]:
for ts in [1,3,10,30,100,300]:
#     regions.calc_peaks(ts, z_th=4, verbose=True, smooth=None)
    regions.calc_peaks(ts)

In [ ]:
# ALLPEAKS = []
fig, axs = plt.subplots(2,3, figsize=(13,7), sharex=True, sharey=True)
for ts,ax in zip([1,3,10,30,100,300], axs.flat):
    df = regions.peaks["%g"%ts].copy()
    ax.hexbin(df["peak half-width [s]"], df["peak height [z-score]"], yscale="log", xscale="log", mincnt=1, gridsize=30, bins="log")
#     df["ts"] = ts
#     ALLPEAKS += [df]

In [ ]:
ts = 10
pdf = regions.peaks["%g"%ts]

In [ ]:
# %matplotlib inline
%matplotlib notebook

In [ ]:
# fitting of trace
fig, ax = plt.subplots(1,1,figsize=(9,4), sharex=True)
ix = regions.df.sort_values("zInterest", ascending=False).index[0]    
y  = regions.df.loc[ix,"trace"].copy()
ys =regions.df.loc[ix,"slower_%g"%ts].copy()
t  = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()
dtt = tt[1]-tt[0]
nrb = int(dtt/regions.Freq)
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
dt = dtt/max(1,nrb)
ax.plot(t, y, lw=.6)
ax.plot(tt, ys, lw=.8)
plt.tight_layout()

ishow = 0
for _,p in pdf.query(f"roi=={ix}").iterrows():
    is0 = np.searchsorted(tt, p.t0-p["peak half-width [s]"]*.5)
    i0  = np.searchsorted(t , p.t0-p["peak half-width [s]"]*.5,)
    ie  = np.searchsorted(t , p.t0+p["peak half-width [s]"]*2+1)
    ycur = y[i0:ie]
    tcur = t[i0:ie]
    imax = np.argmax(ycur)
#     ax.plot(tcur,ycur,"C2.", mfc="none")
    ax.plot(tcur[imax],ycur[imax],"ko")
    
    r = guessDecayPars(ycur[imax-1:ie])[-1]
    pars0 = Parameters(decay=dt/r,
                       half_width=p["peak half-width [s]"]*.5,
                       amplitude=ycur[imax-1:imax+1].mean()-ys[is0],
                       loc=p.t0+p["peak half-width [s]"]*.2,
                       offset=ys[is0],
                       end=ys[is0]
                      )

    y0 = myShape(tcur, *pars0)
    plt.plot(tcur, y0,"grey",lw=.7)
    try:
        opt = minimize(toMin, np.array(pars0), args=(tcur,ycur))
    except:
        continue
    yf,bkg = myShape(tcur,*opt.x, bkgsum=False)
    y[i0:ie][np.isfinite(yf)] -= yf[np.isfinite(yf)]
    plt.plot(tcur, yf+bkg, "k")
    plt.plot(tcur, y[i0:ie], "C1",lw=.5)
    
    if ishow==0:
        tmin = t[i0]-10
    ishow += 1
    if ishow>10: break
plt.xlim(tmin, tcur[-1]+10)

In [ ]:
x = y[i0:ie]+yf
x -= x[:10].mean()
plt.plot(tcur,x)
plt.axvline(p.t0)
plt.plot(tcur,myLogNorm(tcur,Ampl=1,loc=p.t0,s=1,scale=1))
plt.plot(tcur,myLogNorm(tcur,Ampl=1,loc=p.t0,s=1,scale=1.003))
plt.plot()

In [ ]:
tr = np.linspace(0,10,1000)
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=1,scale=1))
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.1,scale=1))
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=1,scale=2))
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=2))
plt.grid()

In [ ]:
ss = np.logspace(-1,.5,11)
for i,s in enumerate(ss):
    plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=s,scale=1, reshift=True), c=plt.cm.rainbow(i/len(ss)))
# plt.xlim(0,3)
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
plt.yscale("log")
plt.ylim(1e-10,2)

In [ ]:
ss = np.logspace(-1,.5,11)
for i,s in enumerate(ss):
    plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=s), c=plt.cm.rainbow(i/len(ss)))
# plt.xlim(0,3)
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
# plt.yscale("log")
# plt.ylim(1e-10,2)

In [ ]:
hw = .7
plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,scale=2*hw))
plt.xlim(0,3)
plt.axhline(.5)
# plt.plot(tr,myLogNorm(tr,Ampl=1,loc=1,s=.2,scale=1),"k--")
# plt.yscale("log")
# plt.ylim(1e-10,2)

In [ ]:
LNParameters = namedtuple("Parameters", ['loc', 'scale', 's', 'amplitude', 'offset', 'end'])

In [ ]:
fitPars = pd.DataFrame(columns=['loc', 'scale', 's', 'amplitude', 'offset', 'end'])

In [ ]:
# fitting of trace with logNorm
fig, ax = plt.subplots(1,1,figsize=(9,4), sharex=True)
ix = regions.df.sort_values("zInterest", ascending=False).index[0]    
y  = regions.df.loc[ix,"trace"].copy()
ys = regions.df.loc[ix,"slower_%g"%ts].copy()
t  = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()

dtt = tt[1]-tt[0]
nrb = int(dtt/regions.Freq)
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
dt = dtt/max(1,nrb)
ax.plot(t, y, lw=.6)
ax.plot(tt, ys, lw=.8)
plt.tight_layout()

ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    is0 = np.searchsorted(tt, p.t0-p["peak half-width [s]"]*.5)
    i0  = np.searchsorted(t , p.t0-p["peak half-width [s]"]*.5,)
    ie  = np.searchsorted(t , p.t0+p["peak half-width [s]"]*2+1)
    ycur = y[i0:ie]
    tcur = t[i0:ie]
    imax = np.argmax(ycur)
#     ax.plot(tcur,ycur,"C2.", mfc="none")
    ax.plot(tcur[imax],ycur[imax],"ko")
#     r = guessDecayPars(ycur[imax-1:ie])[-1]
    pars0 = LNParameters(s=.2,
                         scale=p["peak half-width [s]"]*2,
                         amplitude=ycur[imax-1:imax+1].mean()-ys[is0],
                         loc=p.t0+p["peak half-width [s]"]*.2,
                         offset=ys[is0],
                         end=ys[is0]
                      )

    y0 = myLogNorm(tcur, *pars0)
    plt.plot(tcur, y0,"grey",lw=.7)
    opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm), bounds=[(pars0[j],pars0[j]) if j==2 else (None,None) for j in range(len(pars0))])
    opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm), bounds=[(pars0[j]*.5,pars0[j]*2) if j==2 else (None,None) for j in range(len(pars0))])
    pars = LNParameters(*opt.x)
#     try:
#         opt = minimize(toMin, np.array(pars0), args=(tcur,ycur,myLogNorm))
#     except:
#         continue
    yf,bkg = myLogNorm(tcur,*opt.x, bkgsum=False)
    y[i0:ie][np.isfinite(yf)] -= yf[np.isfinite(yf)]
    plt.plot(tcur, yf+bkg, "k")
    plt.plot(tcur, y[i0:ie], "C1",lw=.5)
    fitPars.loc[ip] = opt.x
#     pdf.loc[[ip],"fitPars"] = [tuple(opt.x)]
    if ishow==0:
        tmin = t[i0]-10
    ishow += 1
#     if ishow>10: break
plt.xlim(tmin, tcur[-1]+10)

In [ ]:
fitPars

In [ ]:
LNParameters(*opt.x)

In [ ]:
pdf.loc[[ip],"fitPars"] = [range(len(pars0))]

In [ ]:
plt.plot(tr,myLogNorm(tr,*pars0._replace(loc=1), fillnan=False))

In [ ]:
p["ha"] = 1

In [ ]:
p

In [ ]:
pdf

In [ ]:
p.update(dict(zip(pars0._asdict().keys(), opt.x)))

In [ ]:
plt.plot(y0)

In [ ]:
# fitting of z
ix = regions.df.sort_values("zInterest", ascending=False).index[0]
if "zScore_%g"%ts not in regions.df:
    regions.fast_filter_traces(ts)
    
nrb = int(ts/regions.Freq)
fig, axs = plt.subplots(2,1,figsize=(9,6), sharex=True)

y = regions.df.loc[ix,"trace"].copy()
ys =regions.df.loc[ix,"slower_%g"%ts].copy()
z = regions.df.loc[ix,"zScore_%g"%ts].copy()
t = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
axs[0].plot(t, y, lw=.6)
axs[0].plot(tt, ys, lw=.8)
axs[1].plot(tt,z,"-", lw=.6)
dtt = tt[1]-tt[0]
plt.tight_layout()
# plt.xlim(t[0]-1,2000)

ishow = 0
for _,p in pdf.query(f"roi=={ix}").iterrows():
    i0 = np.searchsorted(tt,p.t0-1,)
    ie = np.searchsorted(tt,p.t0+p["peak half-width [s]"]*2+1)
    zcur = z[i0:ie]
    tcur = tt[i0:ie]
    imax = np.argmax(zcur)
    axs[1].plot(tcur,zcur,"C2.", mfc="none")
    axs[1].plot(tcur[imax],zcur[imax],"ko")
    pars0 = Parameters(decay=dtt/guessDecayPars(zcur[imax-1:ie])[-1],
                       half_width=p["peak half-width [s]"]*.8,
                       amplitude=z[i0:ie].max(),
                       loc=p.t0,
                       offset=0,
                       end=0
                      )

    z0 = myShape(tcur, *pars0)
    plt.plot(tcur, z0,"grey",lw=.7)
    try:
        opt = minimize(toMin, np.array(pars0), args=(tcur,zcur))
    except:
        continue
    zf = myShape(tcur,*opt.x)
    plt.plot(tcur, zf,"k")
    if ishow==0:
        tmin = tt[i0]-10
    ishow += 1
    if ishow>10: break
plt.xlim(tmin, tcur[-1]+10)

In [ ]:
from islets.numeric import guessDecayPars

In [ ]:
iHalf = i0+int(regions.Freq*(1+p["peak half-width [s]"]))

In [ ]:
pars = Parameters(*opt.x)

In [ ]:
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(tt[i0:ie],z[i0:ie],"o", mfc="none")
plt.plot(tt[iHalf],z[iHalf],"x", mfc="none")

In [ ]:
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars),"k")
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(half_width=pars.half_width/2)))
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(decay=pars.decay*5)))
plt.plot(tt[i0:ie], myShape(tt[i0:ie], *pars._replace(decay=pars.decay*5, half_width=pars.half_width*2.5)))
# plt.ioff()

In [ ]:
1/regions.Freq

### Other playing

In [ ]:
%matplotlib notebook

In [ ]:
ix = 57
coltype = "zScore"
plt.figure(figsize=(10,4))
it = 0
for ts in [1,3,5,10,30]:
    try: t = regions.showTime["%g"%ts]
    except: t = regions.time
    y = regions.df.loc[ix,"%s_%g"%(coltype,ts)]
    plt.plot(t, y+10*it,lw=.6)
    it += 1

In [ ]:
from islets.utils import autocorr

In [ ]:
plt.figure(figsize=(5,4))
for ts in [1,3,5,10,30]:
    try: t = regions.showTime["%g"%ts]
    except: t = regions.time
    dt = np.diff(t).mean()
    x = np.arange(int(30/dt))
    y = autocorr(regions.df.loc[ix,"%s_%g"%(coltype,ts)], x)
    c = plt.plot(x*dt,y,label=ts)[0].get_color()
    plt.plot([-1,x[np.argmin(y)]*dt],[y.min(), y.min()],c=c)
    plt.plot(x[np.argmin(y)]*dt,y.min(),"o",c=c, mfc="w")
plt.legend()

In [ ]:
ts = 3
regions.calc_peaks(ts)
pdf = regions.peaks["%g"%ts]

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
nrb = 10
plt.figure(figsize=(10,4))
y = rebin(regions.df.loc[ix,"trace"],nrb).copy()
t = rebin(regions.time, nrb)
plt.plot(t, y, lw=.6)
ishow = 0
for _,p in pdf.query(f"roi=={ix}").iterrows():
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+4,)
#     break

    offset = y[i0]
#     scale = p["peak half-width [s]"]
    a = y[i0:ie].max()-offset
#     yf = myLogNorm(t,
#                       Ampl=a,
#                       loc=p.t0, 
#                       scale=scale,
#                        s=1,
#                       offset=offset)
    yf = myShape(t,loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset)
    plt.plot(t, yf,"k")
    plt.plot(t, y-yf+offset,"grey")
    ishow += 1
    if ishow>10: break
plt.xlim(t[0],t[ie]+30)

In [ ]:
%matplotlib inline

In [ ]:
def toMin(x):
    yf_ = myShape(t[i0:ie],*x,fillnan=False)
    return ((yf_-y[i0:ie])**2).sum()

In [ ]:
x0 = np.array([
    2.5,
    p["peak half-width [s]"],
    a,
    p["t0"],
    offset,
])
toMin(x0)

In [ ]:
from scipy.optimize import minimize

In [ ]:
opt = minimize(toMin,x0)

In [ ]:
tr = np.linspace(t[i0],t[ie],1000)

In [ ]:
plt.plot(t[i0:ie],y[i0:ie],"o-", mfc="w")
plt.axvline(p.t0,ls="--")
yshow = myShape(tr      ,loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset, fillnan=False)
yf0    = myShape(t[i0:ie],loc=p.t0,decay=2.5,amplitude=a,half_width=p["peak half-width [s]"],offset=offset, fillnan=False)
yf = myShape(tr,*opt.x, fillnan=False)
plt.plot(tr, yshow,"k")
# plt.plot(t[i0:ie], yf0,"ko", mfc="w")
plt.plot(tr,yf)
# plt.plot(t, y-yf+offset,"grey")
# plt.xlim(t[i0]-2,t[ie]+3)

In [ ]:
def toMin(x, t_, y_):
    yf_ = myShape(t_,*x,fillnan=False)
    return ((yf_-y_)**2).sum()

In [ ]:
%matplotlib notebook

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
nrb = 10
plt.figure(figsize=(10,4))
y = rebin(regions.df.loc[ix,"trace"],nrb).copy()
t = rebin(regions.time, nrb)
plt.plot(t, y, lw=.6)
ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    
#     if ishow>=20: break
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+3,)
    x0 = np.array([
            2.5,                      # dacay
            p["peak half-width [s]"], # half width
            y[i0:ie].max()-y[i0],     # ampl
            p["t0"],                  # loc
            y[i0],                    # offset
            y[i0],                    # end
        ])
    tr = np.linspace(t[i0],t[ie],100)
    y0 = myShape(tr,*x0)
    plt.plot(tr, y0, "b", lw=.7)
    
    try:
        opt = minimize(toMin, x0, args=(t[i0:ie], y[i0:ie]))
        ishow += 1
    except:
         continue
    yf = myShape(tr,*opt.x)
    plt.plot(tr, yf,"C3")
    plt.plot(t[i0:ie], y[i0:ie]-myShape(t[i0:ie],*opt.x)+opt.x[-1],"lightgrey",lw=.5)
#     print (ishow, opt.x)
# plt.xlim(t[0],t[ie]+30)

In [ ]:
# pdf.query(f"roi=={ix}")
ix = 57
ts = .5
if "zScore_%g"%ts not in regions.df:
    regions.fast_filter_traces(ts)
    
nrb = 1
fig, axs = plt.subplots(2,1,figsize=(10,8))

In [ ]:
y = regions.df.loc[ix,"trace"].copy()
z = regions.df.loc[ix,"zScore_%g"%ts].copy()
t = regions.time.copy()
try:
    tt = regions.showTime["%g"%ts].copy()
except:
    tt = regions.time.copy()
if nrb>1:
    y = rebin(y, nrb)
    t = rebin(t, nrb)
axs[0].plot(t, y, lw=.6)
axs[1].plot(tt,z)

In [ ]:
from islets.numeric import power_spectrum

In [ ]:
fqs,pw = power_spectrum(y,regions.Freq/nrb)

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(rebin(fqs[fqs>1.5],20), rebin(np.abs(pw)[fqs>1.5],20))

In [ ]:
ishow = 0
for ip,p in pdf.query(f"roi=={ix}").iterrows():
    
#     if ishow>=20: break
    i0 = np.searchsorted(t,p.t0-1,)
    ie = np.searchsorted(t,p.t0+p["peak half-width [s]"]+3,)
    x0 = np.array([
            2.5,                      # dacay
            p["peak half-width [s]"], # half width
            y[i0:ie].max()-y[i0],     # ampl
            p["t0"],                  # loc
            y[i0],                    # offset
            y[i0],                    # end
        ])
    tr = np.linspace(t[i0],t[ie],100)
    y0 = myShape(tr,*x0)
    plt.plot(tr, y0, "b", lw=.7)
    
    try:
        opt = minimize(toMin, x0, args=(t[i0:ie], y[i0:ie]))
        ishow += 1
    except:
         continue
    yf = myShape(tr,*opt.x)
    plt.plot(tr, yf,"C3")
    plt.plot(t[i0:ie], y[i0:ie]-myShape(t[i0:ie],*opt.x)+opt.x[-1],"lightgrey",lw=.5)
#     print (ishow, opt.x)
# plt.xlim(t[0],t[ie]+30)

In [ ]:
# tr = np.linspace(t[i0],t[ie],100)
y0 = myShape(tr,*x0)

### Check for wiggles

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
th = get_sep_th(regions.df.peakValue.values, log=True, plot=True)

In [ ]:
# x = np.vstack([movie[:,point[0],point[1]] for point in regions.df.sort_values("peakValue",ascending=False).peak[:30]])
x = np.vstack([rebMovie[:,point[0],point[1]] for point in regions.df.sort_values("peakValue",ascending=False).peak[:30]])
# x = np.vstack(regions.df.trace)
# f,p = power_spectrum(x, fr=metadata.Frequency)
f,p = power_spectrum(x, fr=rebMovie.fr)
f,p = f[f>0.5],np.abs(p[f>0.5])

In [ ]:
n = 1
plt.plot(rebin(f,n),rebin(p,n))

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(x[0])

In [ ]:
i,j = regions.df.sort_values("peakValue",ascending=False).peak.iloc[0]

In [ ]:
plt.scatter(rebMovie[:,i,j],
            rebMovie[:,i+0,j-1],
            c=np.arange(len(rebMovie)),
            s=.4)
plt.gca().set_aspect("equal")

In [ ]:
# f,p = power_spectrum(np.vstack(data[:,0]),fr=metadata.Frequency,mean=False)

In [ ]:
# plt.boxplot(p[:,2:100],positions=f[2:100], showfliers=False);
# plt.plot(f[2:],p.mean(0)[2:],lw=.6)
# plt.xscale("symlog")

In [ ]:
# plt.plot(*sum([(f,pp) for pp in p[:10]],tuple()),lw=.3)

### Experiment with filtering the whole movie

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
th = get_sep_th(im.flatten(),log=True, plot=False)

In [ ]:
r = rs[0]

In [ ]:
r.update(movie)

In [ ]:
r.Freq

In [ ]:
r.fast_filter_traces(4,z_sp=0)

In [ ]:
n = 256
tmp = r.sosFilter.run(r.movie[:,:n,:n].reshape((r.movie.shape[0],-1)).T).T.reshape((-1,n,n))

In [ ]:
n1,n2 = int(.55*len(tmp)), int(.551*len(tmp))
print (r.time[n2]-r.time[n1])

In [ ]:
# show_movie(tmp,)

In [ ]:
np.percentile(tmp[n1:n2].flatten(),5)

In [ ]:
tmp[n1:n2].min()

In [ ]:
zMovie = (rebMovie[n1:n2]-tmp[n1:n2])/np.maximum(.04,tmp[n1:n2])**.5

In [ ]:
plt.hist(zMovie.flatten(),100,log=True);

In [ ]:
z = 1.5
im = (zMovie>z).sum(0)-(zMovie<-z).sum(0)
plt.imshow(im,cmap="bwr", vmin=-np.abs(im).max(), vmax=np.abs(im).max())

In [ ]:

# show_movie(
#     zMovie,
#     NTimeFrames=n2-n1,
#     log=False, 
#     autoadjust=False,
# #     out="save",
#     cmapArgs={"cmap":"bwr","vmin":-3,"vmax":3}
#           )

In [ ]:
regions = Regions(rebMovie, diag=True, gSig_filt=(3,5))

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(13,6))
for k,ax in zip(["mean","diff_std"],axs):
    ax.imshow(regions.statImages[k], norm=LogNorm())
    regions.plotEdges(ax=ax,image=False)
    regions.plotPeaks(ax=ax,image=False)

In [ ]:
regions.update(movie)

In [ ]:
regions.fast_filter_traces(10)

In [ ]:
n = 10
tmp = regions.sosFilter.run(regions.movie[:,:n,:n].reshape((regions.movie.shape[0],-1))).reshape((-1,n,n))

In [ ]:
from islets.utils import show_movie

In [ ]:
mshow = np.concatenate([
    rebin(movie[:,:n,:n],1200),
    rebin(tmp,1200),
    rebin(movie[:,:n,:n]-tmp,1200)
])

In [ ]:
mshow  = cmovie(mshow,fr=movie.fr)

In [ ]:
show_movie(mshow,
#           additionalPlot=lambda ax_: regions.plotEdges(ax=ax_, image=False),
#           log=False, 
          figScale=1,
          NTimeFrames=len(mshow),
#           out="save"
         )

In [ ]:
regions.calcTraces()

In [ ]:
regions.detrend_traces()

In [ ]:
regions.infer_gain(plot=True)

In [ ]:
regions.gain

In [ ]:
regions.gain = 1

In [ ]:
ts = 4
regions.fast_filter_traces(ts)

In [ ]:
x = np.vstack(regions.df["zScore_%g"%ts])
# x = rebin(x,100,1)
plt.imshow(x>3, aspect="auto")

In [ ]:
regions.df.columns

In [ ]:
regions.fast_filter_traces(10)